<a href="https://colab.research.google.com/github/scastroplus/queersafe/blob/main/Agente_LGBT%2BGemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Célula 1: Instalação e Atualização das Bibliotecas (google-genai)

print("Iniciando instalação e atualização das bibliotecas (modo detalhado)...")
# Instalamos E ATUALIZAMOS as bibliotecas principais.
# Removido googletrans, pois a seção de links será removida.
!pip install --upgrade google-genai requests beautifulsoup4
print("Instalação/Atualização concluída. Verifique a saída acima para erros ou avisos.")

Iniciando instalação e atualização das bibliotecas (modo detalhado)...
Instalação/Atualização concluída. Verifique a saída acima para erros ou avisos.


In [9]:
# Célula Extra: Verificar Versão da Biblioteca google-genai

print("Verificando a versão da biblioteca google-genai...")
try:
    # Usa pip show para obter informações sobre o pacote instalado
    !pip show google-genai
except Exception as e:
    print(f"Erro ao verificar a versão: {e}")
    print("Certifique-se de que a Célula 1 foi executada corretamente para instalar a biblioteca.")

Verificando a versão da biblioteca google-genai...
Name: google-genai
Version: 1.15.0
Summary: GenAI Python SDK
Home-page: https://github.com/googleapis/python-genai
Author: 
Author-email: Google LLC <googleapis-packages@google.com>
License: Apache-2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: anyio, google-auth, httpx, pydantic, requests, typing-extensions, websockets
Required-by: google-cloud-aiplatform


In [10]:
# CÉLULA 2 - Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [11]:
# Célula 3 Modificada: Define o modelo (Sem criar objeto Client)

# Importamos a biblioteca genai.
from google import genai

print("Definindo o ID do modelo a ser usado...")

# >> USANDO O NOME DO MODELO ESPECÍFICO QUE VOCÊ QUER TESTAR <<
# Certifique-se de que este nome está correto e disponível para sua chave de API.
MODEL_ID = "gemini-2.5-flash-preview-04-17" # Exemplo. Use o nome que você verificou.
# Se você quiser testar o 'gemini-2.0-flash', mude aqui: MODEL_ID = "gemini-2.0-flash"

print(f"ID do Modelo configurado para: {MODEL_ID}")

# Não criamos o objeto 'client' nesta versão, pois tentaremos chamar a geração direto do módulo genai.
# client = genai.Client() # <<<<<<<< REMOVIDO

Definindo o ID do modelo a ser usado...
ID do Modelo configurado para: gemini-2.5-flash-preview-04-17


In [16]:
# Célula 4 Corrigida: Função para Interagir com Modelo Gemini e Ferramenta de Busca (Com Instrução de Gênero)

# Importamos os para verificar a API Key.
import os
# Importamos a biblioteca genai.
import google.generativeai as genai

# Assumimos que MODEL_ID já foi definido na Célula 3.

# >>> A função agora aceita o argumento genero <<<
def obter_relatorio_gemini_com_busca(destino, genero):
    """
    Usa um modelo Gemini com a ferramenta de busca para obter informações e gerar um relatório
    direto e conciso, com nuance baseada na pontuação e adaptado ao gênero do usuário,
    incluindo pontuação, justificativa e seções claras.

    Args:
        destino (str): O nome do destino (cidade, estado, país).
        genero (str): A identificação de gênero fornecida pelo usuário.

    Returns:
        str: O texto do relatório gerado pelo modelo Gemini, contendo as seções esperadas
             e a pontuação, ou uma mensagem de erro.
    """
    # Verificamos se a chave da API foi configurada na Célula 2 (verifica variável de ambiente)
    if os.getenv("GOOGLE_API_KEY") is None:
        return f"ERRO: A chave da API Gemini não foi configurada. Não foi possível gerar o relatório para {destino}."

    print(f"\n-> Usando Modelo '{MODEL_ID}' com Ferramenta de Busca para analisar '{destino}' (Com Instrução de Gênero)...\n")

    try:
        print("   Instanciando o modelo...")
        model = genai.GenerativeModel(model_name=MODEL_ID)


        # Define a instrução para adaptação de gênero
        # Esta instrução será incluída no prompt principal.
        prompt_gender_instruction = f"""
        A pessoa que solicitou este relatório se identifica com o seguinte gênero: "{genero}".
        Ao gerar o conteúdo das seções do relatório (Justificativa da Nota, Resumo Geral da Situação, Alertas de Segurança, Dicas Locais LGBT+), **adapte a linguagem para se adequar a este gênero**:
        - Se o gênero fornecido for "masculino", use pronomes e concordâncias gramaticais no masculino em português (ex: "o viajante", "seguro para ele", "bem recebido").
        - Se o gênero fornecido for "feminino", use pronomes e concordâncias gramaticais no feminino em português (ex: "a viajante", "segura para ela", "bem recebida").
        - **Para qualquer outro gênero** (incluindo "transgênero", "Não-binário", "Agênero", "Gênero fluido", ou se o termo fornecido for ambíguo, plural ou não listado), **utilize uma linguagem neutra em português**. Priorize frases e estruturas que evitem concordâncias de gênero explícitas sempre que possível (ex: "pessoa viajante", "seguro para essa pessoa", "bem recebide"). Se a linguagem neutra for difícil de manter em algumas frases, prefira o uso de termos mais gerais que não impliquem gênero. Evite pronomes específicos de gênero (ele/ela) e utilize alternativas neutras ou o nome da pessoa/gênero, se apropriado ao contexto.

        """


        # Define o prompt principal para o modelo.
        # Incluímos a instrução de gênero dentro deste prompt.
        prompt_text = f"""Você é um agente de IA especializado em segurança de viagens para pessoas LGBT+.
Sua tarefa é pesquisar na web (utilizando a ferramenta Google Search disponível)
informações sobre a situação atual (priorizando os **últimos 6 meses**) para pessoas LGBT+ em \"{destino}\".

Utilize a ferramenta de busca para coletar informações sobre os seguintes tópicos no destino \"{destino}\":
1. Leis e direitos LGBT+ RECENTES.
2. Nível de segurança para pessoas LGBT+, incluindo relatos RECENTES (últimos 6 meses) de violência, assédio ou discriminação.
3. Existência e atividades RECENTES (últimos 6 meses) de grupos extremistas (nazistas, neonazistas, fascistas, extrema direita, etc.) E se há relatos RECENTES (últimos 6 meses) de violência ou ameaças desses grupos contra minorias, incluindo a comunidade LGBT+.
4. Aceitação social e cultural RECENTE da comunidade LGBT+, incluindo atitudes gerais em relação a demonstrações públicas de afeto.
5. Informações sobre a comunidade LGBT+ local, espaços seguros (bares, clubes, ONGs, centros, eventos RECENTES).

Após coletar e analisar as informações encontradas (especialmente focando na RECÊNCIA), gere um relatório **DIRETO E CONCISO**.

**Crucialmente:** Ao descrever desafios ou problemas (como reações a demonstrações públicas de afeto), sempre **contextualize-os claramente dentro do panorama geral de segurança e aceitação no destino**.
- Se a **Pontuação de Segurança LGBT-Friendly** for alta (4.0 ou mais), minimize o foco em incidentes isolados ou desafios menores, enfatizando a segurança e aceitação geral. Mencione os desafios apenas como ressalvas menores dentro do contexto positivo.
- Se a pontuação for baixa, detalhe os riscos e desafios com mais ênfase, explicando por que a nota é baixa.

{prompt_gender_instruction}  # <<< Incluindo a instrução de gênero aqui

O relatório DEVE conter as seguintes seções, usando os títulos exatos em negrito e duas estrelas `**`, e usando as informações coletadas:

**Pontuação de Segurança LGBT-Friendly:**
Avalie a segurança e o acolhimento em uma escala **numérica de 0.0 a 5.0**.
- 0.0: Extremamente perigoso, criminalizado com pena severa, alta hostilidade.
- 1.0: Muito arriscado, legalmente restritivo, forte discriminação.
- 2.0: Arriscado, poucas/nenhumas proteções, discriminação social presente.
- 3.0: Algumas preocupações, seguro com precauções; algumas proteções/aceitação.
- 4.0: Seguro e acolhedor na maior parte, boas proteções legais/aceitação.
- 5.0: Totalmente seguro, direitos plenos, sociedade acolhedora, cena vibrante.
**Escreva a pontuação numérica clara e destacada logo abaixo do título \"Pontuação de Segurança LGBT-Friendly:\", no formato: Pontuação: X.X/5.**

**Justificativa da Nota:**
Explique CLARAMENTE por que você deu essa pontuação, referenciando as informações encontradas (leis, segurança, grupos extremistas, aceitação, comunidade), **contextualizando desafios dentro da nota geral conforme instruído acima**. O impacto de leis que criminalizam atos LGBT+ (especialmente com pena severa) ou a presença de grupos extremistas com violência deve ser refletido na nota e justificado explicitamente.

**Resumo Geral da Situação:**
Um panorama CONCISO da situação legal, de segurança e social no destino, **mantendo a nuance e ponderação de acordo com a pontuação geral e adaptando a linguagem ao gênero do usuário**.

**Alertas de Segurança:**
Quaisquer cuidados específicos a serem tomados ou riscos a serem observados com base nas informações, listados de forma clara, **adaptando a linguagem ao gênero do usuário**.

**Dicas Locais LGBT+:**
Informações sobre bares, clubes, eventos, ONGs ou bairros conhecidos (se encontrados), listados de forma clara, **adaptando a linguagem ao gênero do usuário**.

Foque na clareza, precisão e na RECÊNCIA das informações ao gerar o relatório e a pontuação. Se a busca não retornar informações recentes relevantes para um tópico (especialmente sobre grupos extremistas e violência), mencione a falta de dados recentes para esse ponto.

Não use emojis de arco-íris no corpo principal do relatório gerado por você; apenas forneça a pontuação numérica, a justificativa clara e o conteúdo das seções no formato solicitado. A conversão para emojis será feita pelo meu código Python.
"""

        print("   Enviando prompt para o modelo e aguardando resposta (isso pode levar um tempo, o modelo está usando a busca)...\n")
        response = model.generate_content(
            contents=[{"role": "user", "parts": [{"text": prompt_text}]}],
            tool_config={"function_calling_config": {"mode": "ANY"}} # Configuração para permitir uso de ferramentas
        )

        # Verifica se a resposta tem texto válido.
        if response and hasattr(response, 'text'):
             # Mantido para debug durante o desenvolvimento. Em uma aplicação real, seria removido.
             print("\n--- Resposta Crua do Modelo (para depuração) ---")
             print(response.text)
             print("-----------------------------------------------\n")
             return response.text # Retorna o texto gerado pelo modelo
        else:
             # Retorna mensagens de erro que seriam exibidas no frontend
             if response and hasattr(response, 'candidates') and response.candidates and response.candidates[0].finish_reason:
                 return f"Não foi possível gerar o relatório. Motivo da finalização do modelo: {response.candidates[0].finish_reason}. Conteúdo de segurança ou outro problema."
             elif response:
                 return f"Não foi possível gerar o relatório. O modelo retornou uma resposta inesperada ou vazia para '{destino}'. Resposta raw: {response}"
             else:
                 return f"Não foi possível gerar o relatório. Resposta do modelo foi None."


    except Exception as e:
        # Retorna mensagem de erro técnico que seria exibida no frontend
        return f"ERRO: Não foi possível gerar o relatório para '{destino}' devido a um problema na interação com a API Gemini/busca: {e}"

In [17]:
# Célula 5 Final: Função para Exibir Relatório Final (Extração Robusta, Emojis e Ordem Final Sem Links)
# Esta célula não foi alterada nesta rodada de modificações.

import re # Importamos re para extrair informações
import math # Importamos math para arredondamento
from datetime import datetime # Importamos datetime
# Removida a importação de googletrans

def extrair_secao(texto, titulo_secao):
    """
    Extrai o texto de uma seção específica do relatório do modelo, dado o título da seção.

    Args:
        texto (str): O texto completo do relatório do modelo.
        titulo_secao (str): O título exato da seção a ser extraída (ex: "**Justificativa da Nota:**").

    Returns:
        str: O conteúdo da seção encontrada, ou uma string vazia se a seção não for encontrada.
    """
    # A regex procura pelo título da seção (com ou sem ':'), captura o conteúdo até o próximo título em negrito ou até o fim.
    # Usa re.escape para tratar caracteres especiais no título.
    # Adicionado \s* para flexibilidade de espaço após o título e possível ':'.
    # A lookahead (?=\n\*\*) busca a próxima seção que começa com ** no início de uma nova linha.
    # O .*? captura qualquer caractere (incluindo newlines por causa de re.DOTALL) de forma não gananciosa.
    # Adicionado (?:(?=\n\*\*)|$) para garantir que a seção termine antes do próximo título ** ou no fim da string $.
    # Note: A regex de extração continua usando os títulos como pedidos no prompt da Célula 4 (ex: "**Justificativa da Nota:**").
    # A alteração do texto do cabeçalho é apenas na exibição.
    regex = rf'{re.escape(titulo_secao)}:?\s*(.*?)(?:(?=\n\*\*)|$)'
    match = re.search(regex, texto, re.DOTALL)
    if match:
        # Retorna o conteúdo capturado, removendo espaços/quebras de linha extras no início/fim
        return match.group(1).strip()
    return "" # Retorna string vazia se a seção não for encontrada


def exibir_relatorio_final(texto_relatorio_do_modelo, destino):
    """
    Exibe o relatório gerado pelo modelo, extrai a pontuação e seções,
    adiciona a classificação com emojis refinados e o disclaimer, e reordena a exibição
    com os novos textos e formatações.

    Args:
        texto_relatorio_do_modelo (str): O texto do relatório gerado pelo modelo Gemini,
                                         que deve conter a pontuação e seções.
        destino (str): O nome do destino (para o título do relatório).
    """
    # >>> Título do Relatório Final Atualizado <<<
    print(f"O que sabemos sobre {destino.capitalize()}\n") # Removidos os '---' e ajustado o texto

    # >>> Rótulo da Data Atualizado <<<
    print(f"Data da busca: {datetime.now().strftime('%Y-%m-%d')}\n")

    # >>> DEBUG: Imprime a resposta crua do modelo para análise <<<
    # Mantido para debug durante o desenvolvimento. Em uma aplicação real, seria removido.
    print(f"DEBUG: Texto completo do modelo recebido para análise:\n---\n{texto_relatorio_do_modelo}\n---\n")

    pontuacao = None
    justificativa_nota = ""
    resumo_geral = ""
    alertas_seguranca = ""
    dicas_locais = ""

    # --- Tenta extrair a pontuação numérica do texto do modelo (BUSCA ULTRA ROBUSTA) ---\n
    # Procura pelo padrão "Pontuação: X.X/5" em QUALQUER lugar do texto do modelo.
    # Usamos uma regex flexível que busca pela palavra "Pontuação" (com ou sem acento), seguida por ":" e o número X.X/5.
    # Isso deve capturar a pontuação mesmo que o modelo varie um pouco na formatação ao redor.
    match_pontuacao = re.search(r'Pontua(?:ção|cao)?:\s*(\d+(?:[.,]\d*)?)\s*/5', texto_relatorio_do_modelo, re.IGNORECASE)

    # >>> Debugging score extraction <<<
    # Mantido para debug. Em uma aplicação real, seria removido.
    print(f"DEBUG: Resultado da busca regex por pontuação (global e robusta): {match_pontuacao}\n") # Print the match object

    pontuacao_str = None
    if match_pontuacao:
        # Se o padrão foi encontrado, extraímos a string do número
        pontuacao_str = match_pontuacao.group(1)
        # Substitui vírgula por ponto para garantir que float() funcione, caso o modelo use vírgula.
        pontuacao_str = pontuacao_str.replace(',', '.')
        # Mantido para debug. Em uma aplicação real, seria removido.
        print(f"DEBUG: String numérica raw extraída: '{pontuacao_str}'\n")

    if pontuacao_str:
        try:
            pontuacao = float(pontuacao_str)
            pontuacao = max(0.0, min(pontuacao, 5.0))
            # Mantido para debug. Em uma aplicação real, seria removido.
            print(f"DEBUG: Pontuação numérica '{pontuacao_str}' extraída e convertida: {pontuacao:.1f}\n")
        except ValueError:
            pontuacao = None
             # Mantido para debug. Em uma aplicação real, seria removido.
            print(f"DEBUG: Erro ao converter a string '{pontuacao_str}' para float.\n")
    else:
        pontuacao = None # Garante que pontuacao é None se a string numérica não foi extraída ou o match falhou
        # Mantido para debug. Em uma aplicação real, seria removido.
        print("DEBUG: Não foi possível extrair a pontuação numérica do texto do modelo.\n")


    # --- Converte a pontuação numérica para emojis de arco-íris (Lógica Refinada) ---\n
    emojis = "🚫" # Default para 🚫 se a pontuação não for extraída ou for 0.0
    if pontuacao is not None: # Só gera emojis se a pontuação foi extraída com sucesso
        if pontuacao > 0.0: # Se a pontuação é > 0, gera emojis de arco-íris
            pontuacao_arredondada = round(pontuacao * 2) / 2.0 # Arredonda para o 0.5 mais próximo

            num_arco_iris_completos = int(pontuacao_arredondada)
            # Verifica se a parte decimal arredondada é 0.5 (considerando imprecisões de float)
            tem_meio_emoji = abs(pontuacao_arredondada - num_arco_iris_completos - 0.5) < 1e-9

            emojis = "🌈" * num_arco_iris_completos # Adiciona emojis completos

            if tem_meio_emoji:
                 emojis += "½🌈" # Adiciona meio emoji se necessário
        # else: emojis já é "🚫" por default para pontuação 0.0

    # --- Exibe a Pontuação e Emojis (PRIMEIRO NA ORDEM) ---\n
    # >>> Rótulo da Pontuação e Mensagem de Indisponibilidade Atualizados <<<
    # Exibe a pontuação e os emojis gerados (ou default)
    if pontuacao is not None:
         print(f"Pontuação LGBT+ Friendly: {pontuacao:.1f}/5 {emojis}\n")
    else:
         # Se a pontuação não foi extraída (pontuacao is None), exibe a mensagem atualizada com o default de emoji
         print(f"Pontuação LGBT+ Friendly: indisponível {emojis}\n") # emojis será 🚫 aqui


    # --- Extrai o conteúdo de cada seção usando a função auxiliar ---\n
    # Extrai o conteúdo de cada seção esperada, usando a regex que procura o título em negrito.
    justificativa_nota = extrair_secao(texto_relatorio_do_modelo, "**Justificativa da Nota:**")
    # Note: O prompt pede "**Resumo Geral da Situação:**", mas o cabeçalho de exibição será "**Visão geral**"
    resumo_geral = extrair_secao(texto_relatorio_do_modelo, "**Resumo Geral da Situação:**")
    alertas_seguranca = extrair_secao(texto_relatorio_do_modelo, "**Alertas de Segurança:**")
    dicas_locais = extrair_secao(texto_relatorio_do_modelo, "**Dicas Locais LGBT+:**")


    # --- Exibe as seções na NOVA ORDEM DESEJADA (Sem Links e Sem Fallbacks) ---
    # Ordem: Pontuação/Emojis -> Resumo Geral -> Justificativa -> Alertas -> Dicas Locais -> Disclaimer

    # 1. Resumo Geral da Situação (SEGUNDO NA ORDEM) - >>> Cabeçalho Atualizado e em Negrito <<<
    if resumo_geral: # Só exibe a seção se o conteúdo foi extraído
         print("\n**Visão geral**\n")
         print(resumo_geral)
    # Removido o else com mensagem de fallback


    # 2. Justificativa da Nota (TERCEIRO NA ORDEM) - >>> Cabeçalho Atualizado e em Negrito <<<
    if justificativa_nota: # Só exibe a seção se o conteúdo foi extraído
         print("\n**Justificativa da nota**\n") # Texto do cabeçalho ajustado para minúscula 'n' conforme pedido
         print(justificativa_nota)
    # Removido o else com mensagem de fallback

    # 3. Alertas de Segurança (QUARTO NA ORDEM) - >>> Cabeçalho Atualizado e em Negrito <<<
    if alertas_seguranca: # Só exibe a seção se o conteúdo foi extraído
         print("\n**Alertas de Segurança**\n")
         print(alertas_seguranca)
    # Removido o else com mensagem de fallback

    # 4. Dicas Locais LGBT+ (QUINTO NA ORDEM) - >>> Cabeçalho Atualizado e em Negrito <<<
    if dicas_locais: # Só exibe a seção se o conteúdo foi extraído
         print("\n**Dicas Locais LGBT+**\n")
         print(dicas_locais)
    # Removido o else com mensagem de fallback


    # Removido o separador final "--- Fim do Relatório ---"

    # >>> Texto do Disclaimer Atualizado <<<
    print("\n\nDISCLAIMER: Este relatório foi gerado por um agente de IA utilizando um modelo Gemini e buscas na web com base em publicações dos últimos 6 meses. A precisão e completude dependem das informações encontradas pela ferramenta de busca e da capacidade do modelo em interpretá-las e sintetizá-las. A pontuação é baseada na interpretação do modelo e das informações encontradas. A situação em um destino pode mudar rapidamente. Sempre faça sua própria pesquisa adicional, consulte fontes oficiais de viagem e avalie os riscos com base nas suas circunstâncias pessoais pessoais antes de viajar.") # Corrigido o "pessoais pessoais" para "circunstâncias pessoais"

In [18]:
# Célula 6: Fluxo Principal do Agente (Com Input de Gênero)

# Importamos a função que obtém o relatório com busca
from __main__ import obter_relatorio_gemini_com_busca
# Importamos a função que exibe o relatório final
from __main__ import exibir_relatorio_final
# Importamos input para interação com o usuário
from builtins import input


def executar_agente_gemini_completo():
    """
    Orquestra a execução completa do agente: pede gênero, pede destino,
    obtém relatório e exibe o resultado.
    """
    # Textos de Boas-Vindas
    print("SEJA BEM VIADE") # Mantendo a alteração que você fez
    print("Este é o RADAR QUEER SEGURO")
    print("A sua ferramenta de busca em IA e Google para encontrar locais seguros para viajar")

    # >>> Novo Input para Gênero <<<
    genero = input("\nDiga com qual gênero você se identifica: ")

    # Rótulo para entrada do usuário
    destino = input("Digite qual cidade, estado ou país você quer conhecer: ") # Removido o \n inicial aqui, pois o anterior já tem.

    print(f"\nSolicitando relatório para: {destino}")

    # Obtém o relatório do modelo (chama a função da Célula 4)
    # >>> Passando a variável genero para a Célula 4 <<<
    relatorio = obter_relatorio_gemini_com_busca(destino, genero)

    if relatorio.startswith("ERRO:"):
        # Exibe a mensagem de erro retornada pela Célula 4
        print("\nHouve um erro ao obter o relatório:") # Mantido este cabeçalho para o erro
        print(relatorio) # Exibe a mensagem de erro específica retornada pela Célula 4
    else:
        # Processa e exibe o relatório final (chama a função da Célula 5)
        print("\nProcessando e exibindo relatório...") # Mantida mensagem de processamento
        exibir_relatorio_final(relatorio, destino)

# Executa o agente completo ao rodar a célula
executar_agente_gemini_completo()

SEJA BEM VIADE
Este é o RADAR QUEER SEGURO
A sua ferramenta de busca em IA e Google para encontrar locais seguros para viajar

Diga com qual gênero você se identifica: Travesti
Digite qual cidade, estado ou país você quer conhecer: Indonésia

Solicitando relatório para: Indonésia

-> Usando Modelo 'gemini-2.5-flash-preview-04-17' com Ferramenta de Busca para analisar 'Indonésia' (Com Instrução de Gênero)...

   Instanciando o modelo...
   Enviando prompt para o modelo e aguardando resposta (isso pode levar um tempo, o modelo está usando a busca)...


--- Resposta Crua do Modelo (para depuração) ---
Olá! Compreendi que você se identifica como Travesti e precisa de um relatório de segurança de viagens para pessoas LGBT+ na Indonésia, com foco nos últimos 6 meses e linguagem adaptada. Vou pesquisar e gerar o relatório solicitado.

Utilizarei a ferramenta de busca para coletar as informações mais recentes sobre leis, segurança, extremismo, aceitação social e comunidade local para pessoas L